In [1]:
from dataclasses import dataclass
from typing import *
import warnings

### Возможно, не идеальное решение - все же классы работают не только обмениваясь обмениваясь базовыми типами через интерфейсы, а зависят друг от друга, что может усложнить рефакторинг. С другой стороны, это позволяет инкапсулировать логику работы внутрь классов согласно ТЗ и позволить просто класть операции в стек по ходу вызовов. 

### Единственная функция, не реализованная в логике классов - добавление в стек [redo] при взятии из [undo]. Предполагается имплементация в основном коде программы и вызове операций. 

Можно было бы вызывать и внутри push в "undo_Stack", но не хотелось бы добавлять зависимость одного класса от другого и добавлять параметры и так довольно перегруженным функциям.

In [2]:
class Stack():
    """
    An abstract class for all the stack operations
    
    (Could have inhererited from list, but it has massive functional overhead in terms of methods
    from which we basically need a few")
    """
    
    def __init__(self) -> None:

        self.__stack = []

        return

    def __iter__(self) -> Iterable:
        '''
        Return stack as iterable
        '''
        for item in self.__stack:
            yield item
        return

    def __len__(self):
        """
        Get length of the current stack
        """        
        return len(self.__stack)

    def __repr__(self) -> str:
        return f"{self.__stack}"

    def __getitem__(self, index: int) -> Any:
        return self.__stack[index]

    def size(self) -> int:
        return len(self.__stack)


    def pop(self, index: Optional[int] = -1) -> Any:
        '''
        Take top item (self-explanatory)
        '''
        if len(self.__stack) == 0:
            raise IndexError("Can't pop from empty list")

        return self.__stack.pop(index)

    
    def push(self, element: Any, position: Optional[int] = -1) -> None:
        '''
        Add element to stack
        '''
        if not isinstance(position, int):
            raise ValueError("Position must be of type integer")

        if position >= len(self.__stack):
            raise ValueError("Position index out of range")

        if position != -1:
            return self.__stack.insert(position, element)

        return self.__stack.append(element)
        
            
    
    def clear(self) -> None:
        '''
        Empty the stack
        '''
        return self.__stack.clear()


class undo_Stack(Stack):

    def pop(self, text: Text, index: Optional[int] = -1) -> _Operation:
        '''
        Apply changes when poping from stack to revert changes
        '''
        item = super().pop(index)

        item.redo(text)

        return item


class redo_Stack(Stack):

    def pop(self, text: Text, index: Optional[int] = -1) -> _Operation:
        '''
        Apply changes when poping from stack
        '''
        item = super().pop(index)

        item.__call__(text)

        return item


class Text:
    """
    Represent text
    """
    def __init__(self) -> None:

        self.text = ""
        return 

    def __len__(self) -> int:
        return len(self.text)
    
    def __repr__(self) -> str:
        return self.text
    
    def __str__(self) -> str:
        return self.text

    def size(self) -> int:
        return len(self.text)

    def show(self) -> str:
        return self.__str__()

    def clear(self) -> None:
        self.text = ""
        return

    #could have implemented opener and saver as separate classes but YAGNI
    def open_text(self, path: str) -> None:

        if len(self.text) != 0:
            warnings.warn("Non-saved changes were overwritten while opening file")

        with open(path, "r") as f:
            self.text = f.read()

        return

    def save_text(self, path: str) -> None:
        """
        Suboptimal, better rewrite with libpath
        """
        with open(path, "w") as f:
            f.write(self.text)

        return


class Add(_Operation):
    '''
    string: string to be added
    position: position to add from
    '''
    def __call__(self, text: Text) -> Text:
        
        super().__call__(text)
        if self.data.position >= 0:
            text.text = text.text[:self.data.position] + self.data.string + \
            text.text[self.data.position + len(self.data.string):]

            return text

        text.text += self.data.string

    def redo(self, text: Text) -> Text:

        super().__call__(text)

        l_str = len(self.data.string)

        if self.data.position >= 0:
            text.text = text.text[:self.data.position] + text.text[self.data.position + l_str:]
        
            return text

        l_text = len(text.text)
        text.text = text.text[:l_text - l_str]


class Del(_Operation):
    '''
    string: string to be deleted
    position: position to delete from
    '''
    # make position necessary argument
    def __init__(self, string: str, position: int, edited: Optional[str] = "") -> None:

        if position < 0:
            raise ValueError("Position must be positive")
        
        self.data = TextElement(string, position, edited)

    def __call__(self, text: Text) -> Text:
        
        super().__call__(text)

        l_text = len(text.text)
        l_str = len(self.data.string)

        substring = text.text[self.data.position : self.data.position + l_str]

        #sanity check
        if self.data.string != substring:
            raise ValueError("String specified for deletion doesn't match element in text")

        text.text = text.text[:self.data.position] + text.text[self.data.position + l_str:]

        return text

    def redo(self, text: Text) -> Text:

        super().__call__(text)

        l_str = len(self.data.string)

        text.text = text.text[:self.data.position] + self.data.string + text.text[self.data.position:]
        
        return text


## can be implemented as combination of Del and Add, but as for me - this approach is slightly better for reading
class Replace(_Operation):
    '''
    string: substring to be changed
    position: position in text
    edited: substring to be inserted
    '''
    # keep possibility of using replace as delete with zero substitution
    def __init__(self, string: str, position: int, edited: Optional[str] = "") -> None:

        if position < 0:
            raise ValueError("Position must be positive")

        self.data = TextElement(string, position, edited)
    
    def __call__(self, text: Text) -> Text:

        super().__call__(text)

        l_str = len(self.data.string)

        substring = text.text[self.data.position:self.data.position+l_str]

        #sanity check
        if self.data.string != substring:
            raise ValueError("String specified for deletion doesn't match element in text")

        text.text = text.text[:self.data.position] + self.data.edited + text.text[self.data.position + l_str:]

        return text

    def redo(self, text: Text) -> Text:
        
        super().__call__(text)

        l_edit = len(self.data.edited)

        text.text = text.text[:self.data.position] + self.data.string + text.text[self.data.position + l_edit:]

        return text



NameError: name '_Operation' is not defined

### Unfortunately no pytest, just manual testing here

In [ ]:
t = Text()

u_stack = undo_Stack()

op = Add("items ")

for i in range(3):

    op(t)
    u_stack.push(op)

In [ ]:
u_stack

In [ ]:
t

In [ ]:
Del("ms", 15)(t)
u_stack.push(Del("ms", 15))

t

In [ ]:
Del("te", 7)(t)
u_stack.push(Del("te", 7))

t

In [ ]:
Replace("te ", 11)(t)
u_stack.push(Replace("te ", 11))

t

In [ ]:
Replace("i", 10, "abc")(t)
u_stack.push(Replace("i", 10, "abc"))

t

In [ ]:
r_stack = redo_Stack()

In [ ]:
while u_stack:
    
    item = u_stack.pop(t)
    r_stack.push(item)
    print(t)

In [ ]:
r_stack

In [ ]:
while r_stack:

    item = r_stack.pop(t)

    print(t)